In [1]:
#pragma cling add_include_path("/home/ubuntu/IfcOpenShell/build/Linux/x86_64/install/occt-7.3.0/include/opencascade")
#pragma cling add_include_path("/home/ubuntu/IfcOpenShell/build/Linux/x86_64/install/ifcopenshell/include")
#pragma cling add_include_path("/home/ubuntu/IfcOpenShell/build/Linux/x86_64/install/boost-1.59.0")

#pragma cling add_library_path("/home/ubuntu/IfcOpenShell/build/Linux/x86_64/install/occt-7.3.0/lib")
#pragma cling add_library_path("/home/ubuntu/IfcOpenShell/build/Linux/x86_64/install/ifcopenshell/lib")

#pragma cling load("IfcParse.so")
#pragma cling load("IfcGeom.so")
#pragma cling load("libTKHLR.so")

#include <xcpp/xdisplay.hpp>
#include <nlohmann/json.hpp>

#include <ifcparse/IfcFile.h>
#include <ifcgeom_schema_agnostic/IfcGeomIterator.h>
#include <ifcgeom/IfcGeomElement.h>

#include <Standard_Version.hxx>
#include <BRepPrimAPI_MakeBox.hxx>
#include <HLRBRep_Algo.hxx>
#include <HLRBRep_HLRToShape.hxx>
#include <TopExp_Explorer.hxx>
#include <TopExp.hxx>
#include <BRep_Tool.hxx>
#include <TopoDS.hxx>

#include <limits>
#include <array>

In [2]:
struct html {
    std::string s;
    html(const std::string& t) : s(t) {}
};

In [3]:
nlohmann::json mime_bundle_repr(const html& h) {
    auto bundle = nlohmann::json::object();
    bundle["text/html"] = h.s;
    return bundle;
}

In [16]:
{
    auto inf = std::numeric_limits<double>::infinity();
    
    IfcGeom::IteratorSettings settings;
    settings.set(IfcGeom::IteratorSettings::USE_WORLD_COORDS, true);
    settings.set(IfcGeom::IteratorSettings::DISABLE_TRIANGULATION, true);
    
    IfcGeom::entity_filter ef;
    ef.include = true;
    ef.entity_names = {"IfcSlab", "IfcWall"};
    auto filters = std::vector<IfcGeom::filter_t>({ ef });
    
    opencascade::handle<HLRBRep_Algo> hlr(new HLRBRep_Algo);    
    gp_Ax2 transform(gp_Pnt(), gp_Dir(1, 1, 1), gp_Dir(0, 0, -1));
    HLRAlgo_Projector projector(transform);
    
    IfcParse::IfcFile f("duplex.ifc");
    IfcGeom::Iterator<double> it(settings, &f, filters);
    it.initialize();
    do {
        hlr->Add(it.get_native()->geometry().as_compound());
    } while (it.next());

    hlr->Projector(projector);
    hlr->Update();
    hlr->Hide();

    auto hlr_shapes = HLRBRep_HLRToShape(hlr);
    auto projection = hlr_shapes.VCompound();
    
    std::vector<std::array<gp_XYZ, 2> > lines;
    gp_XYZ min(+inf, +inf, +inf);
    gp_XYZ max(-inf, -inf, -inf);
    
    TopExp_Explorer exp(projection, TopAbs_EDGE);
    for (; exp.More(); exp.Next()) {
        TopoDS_Vertex v0, v1;
        TopExp::Vertices(TopoDS::Edge(exp.Current()), v0, v1);
        gp_Pnt p0 = BRep_Tool::Pnt(v0);
        gp_Pnt p1 = BRep_Tool::Pnt(v1);
        lines.push_back({p0.XYZ(), p1.XYZ()});
        for (auto& p : {p0, p1}) {
            for (size_t i = 0; i < 3; ++i) {
                min.ChangeData()[i] = std::min(min.ChangeData()[i], p.XYZ().GetData()[i]);
                max.ChangeData()[i] = std::max(max.ChangeData()[i], p.XYZ().GetData()[i]);
            }
        }
    }
    
    std::stringstream svgs;
    svgs << "<svg height=\"800\" width=\"800\">\n";
  
    const std::array<std::string, 2> XY = {"x", "y"};
    
    for (auto& l : lines) {
        svgs << "<line";
        for (size_t i = 0; i < 2; ++i) {
            for (size_t j = 0; j < 2; ++j) {
                const double v = (l[i].GetData()[j] - min.GetData()[j]) / (max.GetData()[j] - min.GetData()[j]) * 800;
                svgs << " " << XY[1-j] << (i+1) << "=\"" << v << "\"";
            }
        }
        svgs << "style=\"stroke:black\" />\n";
    }
    
    svgs << "</svg>\n";
    html h(svgs.str());
    xcpp::display(h);
}